In [1]:
import urllib.request, json 

In [52]:
#link = "https://statsapi.web.nhl.com/api/v1/teams/8"
link = "https://statsapi.web.nhl.com/api/v1/game/2019020417/feed/live" #all available stats from game
#link = "https://statsapi.web.nhl.com/api/v1/game/2019020417/boxscore" #boxscore

In [53]:
with urllib.request.urlopen(link) as url:
    data = json.loads(url.read().decode())

# Data. Currently for all periods. Need to adjust to be just the first period. 

In [68]:
data["liveData"].keys()
#data["liveData"]["boxscore"].keys()
#data["liveData"]["boxscore"]["teams"].keys()
#data["liveData"]["decisions"]

dict_keys(['plays', 'linescore', 'boxscore', 'decisions'])

In [5]:
data['liveData']['boxscore']['teams']['home']['teamStats']['teamSkaterStats']
#goals, pim, shots, powerPlayPercentage, powerPlayGoals, powerPlayOpportunities, faceoffWinPercentage = data['liveData']['boxscore']['teams']['away']['teamStats']['teamSkaterStats']
away_team_stats = data['liveData']['boxscore']['teams']['away']['teamStats']['teamSkaterStats']
home_team_stats = data['liveData']['boxscore']['teams']['home']['teamStats']['teamSkaterStats']


In [6]:
home_team_categories = list(home_team_stats.keys())
home_team_values = list(home_team_stats.values())
away_team_categories = list(away_team_stats.keys())
away_team_values = list(away_team_stats.values())

In [7]:
home_team_categories = [f"Home_{i}" for i in home_team_categories]
away_team_categories = [f"Away_{i}" for i in away_team_categories]

In [8]:
import pandas as pd
df = pd.DataFrame(columns = home_team_categories + away_team_categories)
df.loc["2019020417"] = home_team_values + away_team_values

In [9]:
df

,Home_goals,Home_pim,Home_shots,Home_powerPlayPercentage,Home_powerPlayGoals,Home_powerPlayOpportunities,Home_faceOffWinPercentage,Home_blocked,Home_takeaways,Home_giveaways,...,Away_pim,Away_shots,Away_powerPlayPercentage,Away_powerPlayGoals,Away_powerPlayOpportunities,Away_faceOffWinPercentage,Away_blocked,Away_takeaways,Away_giveaways,Away_hits
2019020417,3,4,34,33.3,1.0,3.0,41.1,16,10,4,...,8,29,0.0,0.0,1.0,58.9,12,15,7,28


# Need to find the final score and identify the home team as features.

In [64]:
#Not sure I need this
type(data['liveData']) #dictionay
data['liveData'].keys() # plays, linescore, boxscore, decisions
data['liveData']['linescore']
data['liveData']['boxscore']['teams']['home'].keys()
data['liveData']['boxscore']['teams']['away']['teamStats']
data['liveData']['boxscore']['teams']['away']['teamStats']['teamSkaterStats'] #This is good up to here

{'goals': 1,
 'pim': 8,
 'shots': 29,
 'powerPlayPercentage': '0.0',
 'powerPlayGoals': 0.0,
 'powerPlayOpportunities': 1.0,
 'faceOffWinPercentage': '58.9',
 'blocked': 12,
 'takeaways': 15,
 'giveaways': 7,
 'hits': 28}

# Find information for next game

In [113]:
next_game_link = "https://statsapi.web.nhl.com/api/v1/teams/6?expand=team.schedule.next"
with urllib.request.urlopen(next_game_link) as url:
    data = json.loads(url.read().decode())

In [114]:
type(data["teams"]) #list
data["teams"][0].keys() #This is a list of 1, so we need the [0] to index it.
data["teams"][0]["nextGameSchedule"]["dates"][0]["date"] #This finds the date of the next game
next_game_id = data["teams"][0]["nextGameSchedule"]["dates"][0]["games"][0]["gamePk"] #This is the game identifier

In [115]:
next_game_link = f"https://statsapi.web.nhl.com/api/v1/game/{next_game_id}/feed/live"
with urllib.request.urlopen(next_game_link) as url:
    data = json.loads(url.read().decode())

In [95]:
data.keys()
data["liveData"].keys()
#data["liveData"]["plays"]
#data["liveData"]["boxscore"]
data["liveData"]["linescore"].keys()
data["liveData"]["linescore"] #currentpreiod: 1; currentPreiodOrdinal : '1st'

{'currentPeriod': 1,
 'currentPeriodOrdinal': '1st',
 'currentPeriodTimeRemaining': '10:51',
 'periods': [{'periodType': 'REGULAR',
   'startTime': '2019-12-28T00:08:46Z',
   'num': 1,
   'ordinalNum': '1st',
   'home': {'goals': 0, 'shotsOnGoal': 7, 'rinkSide': 'left'},
   'away': {'goals': 0, 'shotsOnGoal': 5, 'rinkSide': 'right'}}],
 'shootoutInfo': {'away': {'scores': 0, 'attempts': 0},
  'home': {'scores': 0, 'attempts': 0}},
 'teams': {'home': {'team': {'id': 7,
    'name': 'Buffalo Sabres',
    'link': '/api/v1/teams/7',
    'abbreviation': 'BUF',
    'triCode': 'BUF'},
   'goals': 0,
   'shotsOnGoal': 7,
   'goaliePulled': False,
   'numSkaters': 5,
   'powerPlay': False},
  'away': {'team': {'id': 6,
    'name': 'Boston Bruins',
    'link': '/api/v1/teams/6',
    'abbreviation': 'BOS',
    'triCode': 'BOS'},
   'goals': 0,
   'shotsOnGoal': 5,
   'goaliePulled': False,
   'numSkaters': 5,
   'powerPlay': False}},
 'powerPlayStrength': 'Even',
 'hasShootout': False,
 'intermiss

In [122]:
data["liveData"]["linescore"]["currentPeriod"] == 1 and data["liveData"]["linescore"]["currentPeriodTimeRemaining"] == "END" 
print(data["liveData"]["linescore"]["currentPeriod"])
print(data["liveData"]["linescore"]["currentPeriodOrdinal"])
data["liveData"]["linescore"]["currentPeriod"] == 1 and data["liveData"]["linescore"]["currentPeriodTimeRemaining"] == "END" 


1
1st


True

In [123]:
if data["liveData"]["linescore"]["currentPeriod"] == 1 and data["liveData"]["linescore"]["currentPeriodTimeRemaining"] == "END": 
    away_team_stats = data['liveData']['boxscore']['teams']['away']['teamStats']['teamSkaterStats']
    home_team_stats = data['liveData']['boxscore']['teams']['home']['teamStats']['teamSkaterStats']
    home_team_categories = list(home_team_stats.keys())
    home_team_values = list(home_team_stats.values())
    away_team_categories = list(away_team_stats.keys())
    away_team_values = list(away_team_stats.values())
    home_team_categories = [f"Home_{i}" for i in home_team_categories]
    away_team_categories = [f"Away_{i}" for i in away_team_categories]
    import pandas as pd
    df = pd.DataFrame(columns = home_team_categories + away_team_categories)
    df.loc[str(next_game_id)] = home_team_values + away_team_values

In [124]:
df

,Home_goals,Home_pim,Home_shots,Home_powerPlayPercentage,Home_powerPlayGoals,Home_powerPlayOpportunities,Home_faceOffWinPercentage,Home_blocked,Home_takeaways,Home_giveaways,...,Away_pim,Away_shots,Away_powerPlayPercentage,Away_powerPlayGoals,Away_powerPlayOpportunities,Away_faceOffWinPercentage,Away_blocked,Away_takeaways,Away_giveaways,Away_hits
2019020582,0,4,11,0.0,0.0,1.0,57.1,0,4,3,...,2,8,50.0,1.0,2.0,42.9,4,1,1,5
